In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os

load_dotenv()

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"

os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGSMITH_PROJECT"] = "radiohead"
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

In [8]:
# from langchain_community.document_loaders import WebBaseLoader
# docs = WebBaseLoader("https://react.dev/learn/describing-the-ui#your-ui-as-a-tree")
# finalDocs = docs.load()

In [9]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("GROUP2_ANUJ_BHANDARI_0348445_CP1.pdf")
finalDocs = loader.load()

In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
docs = splitter.split_documents(finalDocs)

In [11]:
len(docs)

128

In [12]:
docs

[Document(metadata={'producer': 'xdvipdfmx (20240305)', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-07-24T15:08:47+00:00', 'source': 'GROUP2_ANUJ_BHANDARI_0348445_CP1.pdf', 'total_pages': 62, 'page': 0, 'page_label': 'i'}, page_content='BinHero: An AI-Driven Smart W aste Disposal\nMobile App with Object Detection Motion T racking\nA report submitted in partial fulfilment of the requirements\nfor the award of the degree of\nBachelor\nof\nComputer Science (Hons)\nby\nANUJ BHANDARI\n0358445\nCOMPUTER SCIENCE\nINSTITUTE OF INTERNA TIONAL MANAGEMENT SCIENCE (IIMS)\nAFFILIA TED TO T A YLOR’S UNIVERSITY, MALA YSIA\nKathmandu, Nepal\nApril, 2025'),
 Document(metadata={'producer': 'xdvipdfmx (20240305)', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-07-24T15:08:47+00:00', 'source': 'GROUP2_ANUJ_BHANDARI_0348445_CP1.pdf', 'total_pages': 62, 'page': 1, 'page_label': 'i'}, page_content='CER TIFICA TE\nThis is to certify that Anuj Bhandari, Bachelor (Computer Science), has wor

In [13]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")


In [14]:
from langchain_community.vectorstores import FAISS
db = FAISS.from_documents(docs,embeddings)

In [16]:
# Similarity search
# results = db.similarity_search("JavaScript in JSX with curly braces ", k=1)
query = "Which YOLO is used in BinHero ?"

results = db.similarity_search(query, k=1)

print(results[0].page_content)

Anuj Bhandari: Capstone Project I
Justification for BinHero
• SOR T is a lightweight tracking alogrithm which requires minimal computational
resources and is able to perform eﬀiciently in real-time.This allows it to provide
an immediate user feedback in real time when users perform an action of disposal
without overloading or slowing down.
• DeepSOR T is more computationally heavy algorithm which is deployed on a server.
It can cope with complex tasks with even greater accuracy and robustness (like
multiple users or occlusion) using appearance-based Re-ID.
T able 3.1: Algorithm and Model Selection Summary and Justification
Requirement Solution Reason for Selection
Real-time Object Detec-
tion
YOLO11, YOLO11n Rapid single-pass detection with
high accuracy
Mobile Inference YOLO11n supoorts offline usage via tflite, op-
erate on mid-range mobile phones.
Motion verification for Dis-
posing actions
SOR T, DeepSOR T V erifies object movement into the
bin with a consistent temporal pat-
tern.

In [17]:

llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro", temperature=0.9) # 'temperature' controls the randomness of the model's output; higher values make responses more creative.


In [18]:
## Retrieval Chain, Document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>
 Question: {input}

"""
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n Question: {input}\n\n'), additional_kwargs={})])
| ChatGoogleGenerativeAI(model='models/gemini-2.5-pro', google_api_key=SecretStr('**********'), temperature=0.9, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000002BB65DD4550>, default_metadata=())
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [ ]:
# from langchain_core.documents import Document
# res = document_chain.invoke({
#     "input":"How do curly braces work in JSX?",
#     "context": [Document(page_content="""JavaScript in JSX with curly braces 
# JSX lets you write HTML-like markup inside a JavaScript file, keeping rendering logic and content in the same place. Sometimes you will want to add a little JavaScript logic or reference a dynamic property inside that markup. In this situation, you can use curly braces in your JSX to “open a window” to JavaScript:
# const person = {
#   name: 'Gregorio Y. Zara',
#   theme: {
#     backgroundColor: 'black',
#     color: 'pink'
#   }
# };

# export default function TodoList() {
#   return (
#     <div style={person.theme}>
#       <h1>{person.name}'s Todos</h1>
#       <img
#         className="avatar"
#         src="https://i.imgur.com/7vQD0fPs.jpg"
#         alt="Gregorio Y. Zara"
#       />
#       <ul>
#         <li>Improve the videophone</li>
#         <li>Prepare aeronautics lectures</li>
#         <li>Work on the alcohol-fuelled engine</li>
#       </ul>
#     </div>
#   );
# }


# """)]
# })
# res

'Based on the provided context, you can use curly braces in your JSX to “open a window” to JavaScript. This allows you to add JavaScript logic or reference a dynamic property inside the HTML-like markup.'

In [19]:
retriever = db.as_retriever()


In [20]:
from langchain.chains import create_retrieval_chain
retriever_chain = create_retrieval_chain(retriever,document_chain)

In [21]:
retriever_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002BB65610EC0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n Question: {inpu

In [22]:
response = retriever_chain.invoke({"input": query})
print(response)  # See the full response structure

# If the answer is under the "answer" key:
print(response["answer"])

{'input': 'Which YOLO is used in BinHero ?', 'context': [Document(id='f1ea425f-1e7d-4cb5-8c5a-58fec561c509', metadata={'producer': 'xdvipdfmx (20240305)', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-07-24T15:08:47+00:00', 'source': 'GROUP2_ANUJ_BHANDARI_0348445_CP1.pdf', 'total_pages': 62, 'page': 25, 'page_label': '16'}, page_content='Anuj Bhandari: Capstone Project I\nJustification for BinHero\n• SOR T is a lightweight tracking alogrithm which requires minimal computational\nresources and is able to perform eﬀiciently in real-time.This allows it to provide\nan immediate user feedback in real time when users perform an action of disposal\nwithout overloading or slowing down.\n• DeepSOR T is more computationally heavy algorithm which is deployed on a server.\nIt can cope with complex tasks with even greater accuracy and robustness (like\nmultiple users or occlusion) using appearance-based Re-ID.\nT able 3.1: Algorithm and Model Selection Summary and Justification\nRequireme